In [10]:
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import plotly.express as px
from pyproj import Proj, Transformer
import requests

In [1]:
import pandas as pd
import numpy as np
import re


def transform_date(df, columns: str):
    df[columns] = pd.to_datetime(df[columns])
    # Extraire l'année, le mois et le jour de la semaine
    df['Années'] = df[columns].dt.year
    df['Mois'] = df[columns].dt.month
    df['Heure'] = df[columns].dt.hour
    df['Jour de la semaine'] = df[columns].dt.day_name()
    # Mapper les jours de la semaine en français
    jours_semaine_fr = {
        'Monday': 'LUNDI',
        'Tuesday': 'MARDI',
        'Wednesday': 'MERCREDI',
        'Thursday': 'JEUDI',
        'Friday': 'VENDREDI',
        'Saturday': 'SAMEDI',
        'Sunday': 'DIMANCHE'
    }
    # Mapper les mois en français
    mois_fr = {
        1: 'JANVIER',
        2: 'FEVRIER',
        3: 'MARS',
        4: 'AVRIL',
        5: 'MAI',
        6: 'JUIN',
        7: 'JUILLET',
        8: 'AOUT',
        9: 'SEPTEMBRE',
        10: 'OCTOBRE',
        11: 'NOVEMBRE',
        12: 'DECEMBRE'
    }
    # Remplacer les numéros de mois par leur équivalent en français
    df['Mois'] = df['Mois'].map(mois_fr)
    # Remplacer les noms des jours par leur équivalent en français
    df['Jour de la semaine'] = df['Jour de la semaine'].map(jours_semaine_fr)
    return df

def clean_cell_number(df, columns:str):
    df[columns] = df[columns].astype(str)
    df[columns] = df[columns].str.split(',').str[0]
    df[columns] = df[columns].replace(r'^0693', '262693', regex=True)
    df[columns] = df[columns].replace(r'^0692', '262692', regex=True)
    df[columns] = df[columns].replace(r'^06', '336', regex=True)
    df[columns] = df[columns].replace(r'^07', '337', regex=True)
    df[columns] = df[columns].replace(r'^02', '2622', regex=True)
    return df

def reset_accent(chaine):
    accent_fr = {
    'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e',
    'à': 'a', 'â': 'a', 'ä': 'a',
    'ç': 'c',
    'î': 'i', 'ï': 'i',
    'ô': 'o', 'ö': 'o',
    'ù': 'u', 'û': 'u', 'ü': 'u',
    'ÿ': 'y',
    # Ajout des majuscules
    'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
    'À': 'A', 'Â': 'A', 'Ä': 'A',
    'Ç': 'C',
    'Î': 'I', 'Ï': 'I',
    'Ô': 'O', 'Ö': 'O',
    'Ù': 'U', 'Û': 'U', 'Ü': 'U'
}
    for accent, sans_accent in accent_fr.items():
        chaine = chaine.replace(accent, sans_accent)
    return chaine


def clean_city(df, columns: str):
    df[columns] = df[columns].astype(str)
    df = df.rename(columns={columns: 'VILLE'})
    df['VILLE']= df['VILLE'].str.upper()
    df['VILLE']= df['VILLE'].str.replace("-", " ")
    df['VILLE']= df['VILLE'].str.replace("SAINT", "ST")
    df['VILLE']= df['VILLE'].str.replace("SAINTE", "STE")
    df['VILLE']= df['VILLE'].str.replace("L'", "")
    df['VILLE'] = df['VILLE'].apply(reset_accent)
    df['VILLE'] = df['VILLE'].str.replace(r'\s+', ' ', regex=True).str.strip()
    return df


def convert_date(date_str):
    # Extraire le fuseau horaire avec une expression régulière
    match = re.search(r'UTC([+-]\d)', date_str)
    if match:
        utc_offset = int(match.group(1))  # Récupérer le décalage horaire
    else:
        utc_offset = 0  # Valeur par défaut si aucun fuseau horaire n'est trouvé
    # Retirer le fuseau horaire et convertir en datetime
    date_without_tz = date_str.split(' UTC')[0]  # Retirer ' UTC+X'
    dt = pd.to_datetime(date_without_tz, format='%d/%m/%Y - %H:%M:%S')
    # Ajuster le fuseau horaire à UTC+4
    dt = dt + pd.Timedelta(hours=(4 - utc_offset))  # Ajustement basé sur l'offset
    return dt

# Fonction pour nettoyer les chaînes
def clean_number(number_str):
    # Utiliser une expression régulière pour supprimer le préfixe et le suffixe
    return re.sub(r'^\=\("\s*|\s*"\)$', '', number_str)

def replace_unknown_ville(row):
    reunion_postal_codes = {
    "97400": "ST DENIS",
    "97410": "ST PIERRE",
    "97411": "BOIS DE NEFLES",
    "97412": "BRAS PANON",
    "97413": "CILAOS",
    "97414": "ENTRE DEUX",
    "97416": "LA CHALOUPE",
    "97417": "ST BERNARD",
    "97418": "LA PLAINE DES CAFRES",
    "97419": "LA POSSESSION",
    "97420": "LE PORT",
    "97421": "LA RIVIERE ST LOUIS",
    "97422": "LA SALINE",
    "97423": "LE GUILLAUME",
    "97424": "PITON ST LEU",
    "97425": "LES AVRIONS",
    "97426": "LES TROIS BASSINS",
    "97427": "L'ETANG SALE",
    "97428": "LA NOUVELLE",
    "97429": "PETITE ILE",
    "97430": "LE TAMPON",
    "97431": "LA PLAINE DES PALMISTES",
    "97432": "LA RAVINE DES CABRIS",
    "97433": "SALAZIE - HELL BOURG",
    "97434": "ST GILLES LES BAINS",
    "97435": "BERNICA",
    "97436": "ST LEU",
    "97437": "STE ANNE",
    "97438": "STE MARIE",
    "97439": "STE ROSE",
    "97440": "ST ANDRE",
    "97441": "STE SUZANNE",
    "97442": "BASSE VALLEE",
    "97450": "ST LOUIS",
    "97460": "ST PAUL",
    "97470": "ST BENOIT"
}
    if row['VILLE'] == 'ville inconnue':
        return reunion_postal_codes.get(row['CODE POSTAL'], 'ville inconnue')  # Remplace par la ville correspondante ou garde 'ville inconnue'
    return row['VILLE']


def extract_city(address):
    if isinstance(address, str):  # Vérifie si l'adresse est une chaîne
        # Regex pour capturer la ville après le code postal (5 chiffres)
        match = re.search(r'\d{5}\s+(.*)', address)
        if match:
            return match.group(1).strip()  # Retourne la ville sans espaces superflus
    return None  # Retourne None si aucune correspondance n'est trouvée ou si l'adresse n'est pas une chaîne


In [12]:
import pandas as pd


def preprocess_data(file1):
    # Liste des colonnes attendues
    expected_columns = [
        "Date de début d'appel",
        "MSISDN Abonné",
        "Correspondant",
        "Type de communication",
        "Durée / Nbr SMS",
        "Adresse du relais",
        "IMEI abonné",
        "IMSI abonné"
    ]
    # Lire à nouveau avec usecols
    df = pd.read_csv(file1, header=1, sep=';', encoding='latin1', dtype={"MSISDN Abonné": str, "Correspondant": str, "IMEI abonné": str, "IMSI abonné": str, "Durée / Nbr SMS": str})
    available_columns = df.columns.tolist()
    # Filtrer les colonnes attendues qui sont disponibles
    filtered_columns = list(set(expected_columns) & set(available_columns))
    df = df[filtered_columns]
    rename_dict = {
        "Date de début d'appel": "DATE",
        "MSISDN Abonné": "ABONNE",
        "Type de communication": "TYPE D'APPEL",
        "Durée / Nbr SMS": "DUREE",
        "Adresse du relais": "ADRESSE",
        "IMEI abonné": "IMEI",
        "IMSI abonné": "IMSI",
        "Correspondant": "CORRESPONDANT"
    }
    # Renommer uniquement les colonnes présentes dans le DataFrame
    df.rename(columns={k: v for k, v in rename_dict.items() if k in df.columns}, inplace=True)
    if 'DATE' in df.columns:
        df = transform_date(df, 'DATE')
    if "TYPE D'APPEL" in df.columns:
        df["TYPE D'APPEL"] = df["TYPE D'APPEL"].astype(str)
        df["TYPE D'APPEL"] = df["TYPE D'APPEL"].str.upper()
        df["TYPE D'APPEL"] = df["TYPE D'APPEL"].apply(reset_accent)
    if 'IMEI' in df.columns:
        df['IMEI'] = df['IMEI'].fillna("INDETERMINE")
        df['IMEI'] = df['IMEI'].astype(str)
        df["IMEI"] = df["IMEI"].apply(clean_number)
    if 'IMSI' in df.columns:
        df['IMSI'] = df['IMSI'].fillna("INDETERMINE")
        df["IMSI"] = df["IMSI"].astype('str')
        df['IMSI'] = df['IMSI'].apply(clean_number)
    if 'ABONNE' in df.columns:
        df['ABONNE'] = df['ABONNE'].fillna('')
        df['ABONNE'] = df['ABONNE'].apply(clean_number)
        df = clean_cell_number(df, 'ABONNE')
    if 'CORRESPONDANT' in df.columns:
        df['CORRESPONDANT'] = df['CORRESPONDANT'].fillna("DATA")
        df['CORRESPONDANT'] = df['CORRESPONDANT'].astype(str)
        df['CORRESPONDANT'] = df['CORRESPONDANT'].apply(clean_number)
        df = clean_cell_number(df, 'CORRESPONDANT')
    if 'ADRESSE' in df.columns:
        df['ADRESSE'] = df['ADRESSE'].fillna("INDETERMINE")
        df['ADRESSE'] = df['ADRESSE'].astype(str)
        df['ADRESSE'] = df['ADRESSE'].str.upper()
        df['ADRESSE'] = df['ADRESSE'].apply(reset_accent)
        # Appliquer la fonction pour créer une nouvelle colonne 'Ville'
        df['VILLE'] = df['ADRESSE'].apply(extract_city)
        df = clean_city(df, columns='VILLE')
    df.fillna("INDETERMINE", inplace=True)
    final_rename_dict = {
        "Années": "ANNEE", "Mois": "MOIS", "Heure": "HEURE", "Jour de la semaine": "JOUR DE LA SEMAINE"}
    df.rename(columns={k: v for k, v in final_rename_dict.items() if k in df.columns}, inplace=True)

    return df

In [13]:
df_mt20 = preprocess_data("/home/antoine/telephony_insight_project/streamlit/app/source/models_files/ORRE_MT20.csv")

df_mt20.head()

,DATE,TYPE D'APPEL,CORRESPONDANT,DUREE,IMEI,ADRESSE,IMSI,ABONNE,ANNEE,MOIS,HEURE,JOUR DE LA SEMAINE,VILLE
0,2024-12-05 17:33:49,DATA,DATA,0,356415750069700,ANCIEN CHEMIN DU PHARE 97420 LE PORT,647000122980517,262692293744,2024,DECEMBRE,17,JEUDI,LE PORT
1,2024-12-05 17:33:49,DATA,DATA,81,356415750069700,ANCIEN CHEMIN DU PHARE 97420 LE PORT,647000122980517,262692293744,2024,DECEMBRE,17,JEUDI,LE PORT
2,2024-12-05 17:35:10,DATA,DATA,18,356415750069700,RUE SALINS 97460 SAINT PAUL,647000122980517,262692293744,2024,DECEMBRE,17,JEUDI,ST PAUL
3,2024-12-05 17:35:28,DATA,DATA,0,356415750069700,RUE SALINS 97460 SAINT PAUL,647000122980517,262692293744,2024,DECEMBRE,17,JEUDI,ST PAUL
4,2024-12-05 17:35:28,DATA,DATA,180,356415750069700,RUE SALINS 97460 SAINT PAUL,647000122980517,262692293744,2024,DECEMBRE,17,JEUDI,ST PAUL


In [24]:
df_mt20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Correspondant       208 non-null    object        
 1   Date                208 non-null    datetime64[ns]
 2   Durée               208 non-null    object        
 3   Type d'appel        208 non-null    object        
 4   Abonné              208 non-null    object        
 5   IMSI                208 non-null    object        
 6   Adresse             208 non-null    object        
 7   IMEI                208 non-null    object        
 8   Années              208 non-null    int32         
 9   Mois                208 non-null    object        
 10  Jour de la semaine  208 non-null    object        
 11  Ville               208 non-null    object        
dtypes: datetime64[ns](1), int32(1), object(10)
memory usage: 18.8+ KB


In [7]:
df_mt24 = preprocess_data("/home/antoine/telephony_insight_project/streamlit/app/source/models_files/ORRE_MT24.csv")

df_mt24.head()

,Correspondant,Date,Durée,Type d'appel,Abonné,IMSI,Adresse,IMEI,Années,Mois,Jour de la semaine,Ville
0,262692144008,2023-03-17 12:34:17,1,FR-SMS-ENTR,262692082852,647049700144759,STADE JEAN IVOULA RUE DU STADE DE L'EST 97400 ...,867269036785390,2023,Mars,Vendredi,ST DENIS
1,262692144008,2023-03-17 12:42:29,60,FR-APP-SORT,262692082852,647049700144759,STADE JEAN IVOULA RUE DU STADE DE L'EST 97400 ...,867269036785390,2023,Mars,Vendredi,ST DENIS
2,262692144008,2023-03-17 12:59:00,104,FR-APP-ENTR,262692082852,647049700144759,STADE JEAN IVOULA RUE DU STADE DE L'EST 97400 ...,867269036785390,2023,Mars,Vendredi,ST DENIS
3,262692434962,2023-03-17 13:49:28,207,FR-APP-SORT,262692082852,647049700144759,CENTRE COMMERCIAL STE MARIE DUPARC ROND POINT ...,867269036785390,2023,Mars,Vendredi,STE MARIE
4,262692434962,2023-03-17 13:57:28,54,FR-APP-SORT,262692082852,647049700144759,CENTRE COMMERCIAL STE MARIE DUPARC ROND POINT ...,867269036785390,2023,Mars,Vendredi,STE MARIE


In [23]:
df_mt24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Correspondant       138 non-null    object        
 1   Date                138 non-null    datetime64[ns]
 2   Durée               138 non-null    object        
 3   Type d'appel        138 non-null    object        
 4   Abonné              138 non-null    object        
 5   IMSI                138 non-null    object        
 6   Adresse             138 non-null    object        
 7   IMEI                138 non-null    object        
 8   Années              138 non-null    int32         
 9   Mois                138 non-null    object        
 10  Jour de la semaine  138 non-null    object        
 11  Ville               138 non-null    object        
dtypes: datetime64[ns](1), int32(1), object(10)
memory usage: 12.5+ KB


In [9]:
df_mt20_no_adress = preprocess_data("/home/antoine/telephony_insight_project/streamlit/app/source/models_files/ORRE_MT20_NO_ADRESSE.csv")

df_mt20_no_adress.head()

,Correspondant,Date,Durée,Type d'appel,Abonné,IMSI,IMEI,Années,Mois,Jour de la semaine
0,Data,2023-12-13 00:37:53,3600,DATA,262693774358,647000122833247,865503059975160,2023,Décembre,Mercredi
1,Data,2023-12-13 00:59:16,3600,DATA,262693774358,647000122833247,865503059975160,2023,Décembre,Mercredi
2,Data,2023-12-13 01:37:53,3600,DATA,262693774358,647000122833247,865503059975160,2023,Décembre,Mercredi
3,Data,2023-12-13 01:59:16,3600,DATA,262693774358,647000122833247,865503059975160,2023,Décembre,Mercredi
4,Data,2023-12-13 02:37:53,3600,DATA,262693774358,647000122833247,865503059975160,2023,Décembre,Mercredi


In [11]:
# Compter le nombre de communications par correspondant
def count_corr(df):
    try:
        # Compter le nombre de contacts par correspondant
        correspondant_counts = df['Correspondant'].value_counts().reset_index()
        correspondant_counts.columns = ['Correspondant', 'Nombre de communication']
        # Filtrer pour garder uniquement les correspondants ayant 11 ou 12 caractères
        correspondant_counts = correspondant_counts[correspondant_counts['Correspondant'].str.len().isin([11, 12])]
        total_contacts = correspondant_counts['Nombre de communication'].sum()
        # Calculer le pourcentage et arrondir à un chiffre après la virgule
        correspondant_counts['Pourcentage'] = ((correspondant_counts['Nombre de communication'] / total_contacts) * 100).round(1)
        # Trier par nombre de contacts du plus élevé au plus bas
        correspondant_counts = correspondant_counts.sort_values(by='Nombre de communication', ascending=False)
        return correspondant_counts
    except Exception as e:
        print(f"Erreur lors du comptage des correspondants: {e}")
        return pd.DataFrame()  # Retourner un DataFrame vide en cas d'erreur

# Compter le nombre de communications par IMEI
def count_IMEI(df):
    try:
        imei_counts = df['IMEI'].value_counts().reset_index()
        imei_counts.columns = ['IMEI', 'Nombre de communication']
        return imei_counts
    except Exception as e:
        print(f"Erreur lors du comptage des IMEI: {e}")
        return pd.DataFrame()

# Compter le nombre de communications par IMSI
def count_IMSI(df):
    try:
        imsi_counts = df['IMSI'].value_counts().reset_index()
        imsi_counts.columns = ['IMSI', 'Nombre de communication']
        return imsi_counts
    except Exception as e:
        print(f"Erreur lors du comptage des IMSI: {e}")
        return pd.DataFrame()

# Compter le nombre de types d'appel et créer un graphique en secteurs
def count_phone_type(df):
    try:
        type_appel_counts = df["Type d'appel"].value_counts().reset_index()
        type_appel_counts.columns = ['Type d\'appel', 'Nombre']
        colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']  # Définir les couleurs pour le graphique
        fig = go.Figure(data=[go.Pie(labels=type_appel_counts['Type d\'appel'],
                                       values=type_appel_counts['Nombre'])])
        fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                          marker=dict(colors=colors, line=dict(color='#000000', width=2)))
        return fig  # Retourner la figure Plotly
    except Exception as e:
        print(f"Erreur lors du comptage des types d'appel: {e}")
        return go.Figure()  # Retourner une figure vide en cas d'erreur

# Créer un histogramme global du nombre de communications par jour
def comm_histo_global(df):
    try:
        df['DateOnly'] = df['Date'].dt.date  # Extraire uniquement la date
        daily_counts = df.groupby('DateOnly').size().reset_index(name='Nombre de communications')
        daily_counts['DateOnly'] = pd.to_datetime(daily_counts['DateOnly'])  # Convertir en datetime
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=daily_counts['DateOnly'],
            y=daily_counts['Nombre de communications'],
            marker=dict(line=dict(color='black', width=1))  # Bordure noire autour des barres
        ))
        fig.update_layout(
            title='Nombre de communications par jour',
            xaxis_title='Date',
            yaxis_title='Nombre de communications',
            xaxis=dict(tickformat='%Y-%m-%d'),  # Format des dates sur l'axe x
            showlegend=False  # Masquer la légende si non nécessaire
        )
        return fig
    except Exception as e:
        print(f"Erreur lors de la création de l'histogramme global: {e}")
        return go.Figure()

# Créer un histogramme mensuel du nombre de communications par mois
def comm_histo_monthly(df):
    try:
        fig_monthly = go.Figure()
        fig_monthly.add_trace(go.Histogram(x=df['Mois'].dropna().astype(str),
                                            histfunc='count',
                                            name='Communications par mois',
                                            marker=dict(line=dict(color='black', width=1))))
        fig_monthly.update_layout(title='Nombre de communications par mois',
                                   xaxis_title='Mois',
                                   yaxis_title='Nombre de communications')
        return fig_monthly
    except Exception as e:
        print(f"Erreur lors de la création de l'histogramme mensuel: {e}")
        return go.Figure()

# Créer un histogramme du nombre de communications par jour de la semaine
def comm_histo_weekday(df):
    try:
        # Définir l'ordre des jours de la semaine
        jours_semaine = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']

        # Compter le nombre de communications par jour
        counts = df['Jour de la semaine'].value_counts().reindex(jours_semaine, fill_value=0)

        # Créer l'histogramme
        fig_weekday = go.Figure()
        fig_weekday.add_trace(go.Bar(x=counts.index,
                                      y=counts.values,
                                      name='Communications par jour de la semaine',
                                      marker=dict(line=dict(color='black', width=1))))

        # Mettre à jour la mise en page
        fig_weekday.update_layout(title='Nombre de communications par jour de la semaine',
                                   xaxis_title='Jour de la semaine',
                                   yaxis_title='Nombre de communications',
                                   xaxis=dict(type='category'))  # Assurez-vous que l'axe X est traité comme une catégorie

        return fig_weekday
    except Exception as e:
        print(f"Erreur lors de la création de l'histogramme par jour de la semaine: {e}")
        return go.Figure()

# Créer un histogramme du nombre de communications par heure
def comm_histo_hour(df):
    try:
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir en datetime
        df['Hour'] = df['Date'].dt.hour  # Extraire l'heure
        fig = px.histogram(df, x='Hour', title='Nombre de communications par heure de la journée',
                           labels={'Hour': 'Heure', 'count': 'Nombre de communications'},
                           histnorm='')
        # Ajouter des bordures aux barres
        fig.update_traces(marker=dict(line=dict(color='black', width=1)))
        return fig
    except Exception as e:
        print(f"Erreur lors de la création de l'histogramme par heure: {e}")
        return go.Figure()

# Compter le nombre d'appels par adresse et calculer les pourcentages associés
def adresse_count(df):
    try:
       adresse_counts = df['Adresse'].value_counts().reset_index()
       adresse_counts.columns = ['Adresse', 'Nombre de déclenchement']
       adresse_counts.dropna(axis=0, inplace=True)  # Supprimer les lignes sans adresse
       total_contacts = adresse_counts['Nombre de déclenchement'].sum()
       adresse_counts['Pourcentage'] = ((adresse_counts['Nombre de déclenchement'] / total_contacts) * 100).round(1)
       adresse_counts.sort_values(by='Nombre de déclenchement', ascending=False, inplace=True)
       return adresse_counts
    except Exception as e:
       print(f"Erreur lors du comptage des adresses: {e}")
       return pd.DataFrame()

# Créer un scatter plot basé sur les villes et les dates
def scatter_plot_ville(df):
    try:
       fig = px.scatter(df, x='Date', y='Ville', color='Ville', title='Localisation en fonction de la date')
       return fig
    except Exception as e:
       print(f"Erreur lors de la création du scatter plot des villes: {e}")
       return go.Figure()  # Retourner une figure vide en cas d'erreur


# Liste pour stocker les adresses non trouvées
non_found_addresses = []

# Fonction pour géocoder une adresse via un service externe (API)
def geocode_address_datagouv(address):
    try:
        url = "https://data.geopf.fr/geocodage/search"
        params = {
            'q': address,
            'limit': 1  # Limiter à 1 résultat
        }
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            if data and 'features' in data and len(data['features']) > 0:
                coords = data['features'][0]['geometry']['coordinates']
                return (coords[1], coords[0])  # Retourner (latitude, longitude)
            else:
                non_found_addresses.append(address)  # Ajouter à la liste si non trouvé
                return (None, None)
        else:
            print(f"Erreur lors de la requête : {response.status_code}")
            non_found_addresses.append(address)  # Ajouter à la liste si erreur
            return (None, None)
    except Exception as e:
        print(f"Erreur lors du géocodage de l'adresse '{address}': {e}")
        non_found_addresses.append(address)  # Ajouter à la liste si exception
        return (None, None)


# Créer une carte des adresses pour l'opérateur ORRE.
def carto_adresse_orre(df):
    try:
        fig = px.scatter_map(df,
                            lat="Latitude", lon="Longitude",
                            color="Pourcentage", size="Nombre de déclenchement",
                            hover_name="Adresse",
                            size_max=60, zoom=10,
                            color_continuous_scale=px.colors.sequential.Bluered,
                            map_style="carto-positron")
        return fig
    except Exception as e:
        print(f"Erreur lors de la création de la carte ORRE: {e}")
        return go.Figure()

In [26]:
corr = count_corr(df_mt24)

corr.head()

,Correspondant,Nombre de communication,Pourcentage
0,262692434962,44,55.0
1,262692144008,35,43.8
6,262693017384,1,1.2


In [27]:
imsi = count_IMSI(df_mt24)

imsi.head()

,IMSI,Nombre de communication
0,647049700169491,133
1,647049700144759,5


In [29]:
print(imsi.shape[0])

2


In [34]:
total_days = (df_mt24['Date'].max() - df_mt24['Date'].min()).days
print(f"Nombre total de jours: {total_days}")

Nombre total de jours: 11


In [35]:
fig = px.histogram(df_mt24, x="Date", color = "IMSI", nbins=total_days, title="Nombre de communications par jour")
fig.update_layout(bargap=0.01)
fig.show()